<h1><center>PetFinder: Image Augmentations Master Notebook</center></h1>
                                                      
<center><img src = "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS1Dxr7XNyny7wDQS3i4nhDy3sZamt8bfKrZg&usqp=CAU" width = "750" height = "500"/></center>                                                                          

I have come across a lot of Kaggle Notebooks using a variety of Image Augmentations some of which might feels a bit advanced and difficult to understand in the first go.  
  
The purpose of this notebook is to have the best and important augmentations all at one place. The notebook will be in continuous development as long as I keep on finding interesting stuff to add to it.

<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Contents</center></h2>

> | S.No       |                   Heading                |
> | :------------- | :-------------------:                |         
> |  01 |  [**Competition Overview**](#competition-overview)  |                   
> |  02 |  [**Libraries**](#libraries)                        |  
> |  03 |  [**Global Config**](#global-config)                |
> |  04 |  [**Weights and Biases**](#weights-and-biases)      |
> |  05 |  [**Load Datasets**](#load-datasets)                |
> |  06 |  [**What is Data Augmentation?**](#what-is-data-augmentation)  |
> |  07 |  [**Basic Image Augmentations**](#basic-image-augmentations)   |
> |  08 |  [**Intermediate Image Augmentations**](#intermediate-image-augmentations)   |
> |  09 |  [**Advanced Data Augmentations**](#intermediate-image-augmentations) |
> |  10 |  [**References**](#references)                      |

<div class="list-group" id="list-tab" role="tablist">
<h3 class="list-group-item list-group-item-action active" data-toggle="list" style='background:maroon; border:0; color:white' role="tab" aria-controls="home"><center>If you find this notebook useful, do give me an upvote, it helps to keep up my motivation. This notebook will be updated frequently so keep checking for furthur developments.</center></h3>

<a id="competition-overview"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Competition Overview</center></h2>

Let us have a brief walkthrough about the competition first.

## **<span style="color:orange;">Description</span>**


In this competition, you’ll analyze raw images and metadata to predict the “Pawpularity” of pet photos. You'll train and test your model on PetFinder.my's thousands of pet profiles. Winning versions will offer accurate recommendations that will improve animal welfare.
  
If successful, your solution will be adapted into AI tools that will guide shelters and rescuers around the world to improve the appeal of their pet profiles, automatically enhancing photo quality and recommending composition improvements. As a result, stray dogs and cats can find their "furever" homes much faster. With a little assistance from the Kaggle community, many precious lives could be saved and more happy families created..
  
---

## **<span style="color:orange;">Evaluation Metric</span>**

Submissions are scored on the root mean squared error.

---

<a id="libraries"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Libraries</center></h2>

We are installing all dependencies at one place.

In [ ]:
%%sh
pip install -q albumentations
pip install -q --upgrade wandb

In [ ]:
import gc
import os
import glob
import sys
import cv2
import imageio
import joblib
import math
import random
import wandb
import math

import numpy as np
import pandas as pd

from scipy.stats import kstest

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.image as mpimg

from PIL import Image

from statsmodels.graphics.gofplots import qqplot

plt.rcParams.update({'font.size': 18})
plt.style.use('fivethirtyeight')

import seaborn as sns
import matplotlib

from termcolor import colored

from multiprocessing import cpu_count
from tqdm.notebook import tqdm
from sklearn.model_selection import StratifiedKFold
from scipy.stats import pearsonr

import torch
import transformers
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import torchvision
import torchvision.transforms as transforms

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import r2_score, mean_squared_error

from albumentations import (
    HorizontalFlip, VerticalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine, RandomResizedCrop,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose, Normalize, Cutout, CoarseDropout, ShiftScaleRotate, CenterCrop, Resize
)
from albumentations.pytorch import ToTensorV2

# Core layers
from keras.layers \
    import Activation, Dropout, Flatten, Dense, Input, LeakyReLU

# Normalization layers
from keras.layers import BatchNormalization

# Merge layers
from keras.layers import concatenate, multiply

# Embedding Layers
from keras.layers import Embedding

# Keras models
from keras.models import Model, Sequential

# Keras optimizers
from tensorflow.keras.optimizers import Adam, RMSprop, SGD

import warnings
warnings.simplefilter('ignore')

# Activate pandas progress apply bar
tqdm.pandas()

In [ ]:
# Wandb Login
import wandb
wandb.login()

<a id="global-config"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Global Config</center></h2>

We shall declare all the required configurations in the `config` class so that it comes in handy throughout the code.

In [ ]:
class config:
    DIRECTORY_PATH = "../input/petfinder-pawpularity-score"
    TRAIN_FOLDER_PATH = DIRECTORY_PATH + "/train"
    TRAIN_CSV_PATH = DIRECTORY_PATH + "/train.csv"
    TEST_CSV_PATH = DIRECTORY_PATH + "/test.csv"
    
    SEED = 42

In [ ]:
# wandb config
WANDB_CONFIG = {
     'competition': 'PetFinder', 
              '_wandb_kernel': 'neuracort'
    }

We set the seed to a standard value. 

In [ ]:
def set_seed(seed=config.SEED):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
set_seed()

<a id="weights-and-biases"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Weights and Biases</center></h2>

<center><img src = "https://i.imgur.com/1sm6x8P.png" width = "750" height = "500"/></center>        

**Weights & Biases** is the machine learning platform for developers to build better models faster.

You can use W&B's lightweight, interoperable tools to

- quickly track experiments,
- version and iterate on datasets,
- evaluate model performance,
- reproduce models,
- visualize results and spot regressions,
- and share findings with colleagues.
  
Set up W&B in 5 minutes, then quickly iterate on your machine learning pipeline with the confidence that your datasets and models are tracked and versioned in a reliable system of record.

In this notebook I will use Weights and Biases's amazing features to perform wonderful visualizations and logging seamlessly.

---

<a id="load-datasets"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Load Datasets</center></h2>

> ### **<span style="color:orange;">Training Data</span>**
> - train/ - Folder containing training set photos of the form {id}.jpg, where {id} is a unique Pet Profile ID.
> - train.csv - Metadata (described below) for each photo in the training set as well as the target, the photo's Pawpularity score. The Id column gives the photo's unique Pet Profile ID corresponding the photo's file name.
> 
> ### **<span style="color:orange;">Example Test Data</span>**
> In addition to the training data, we include some randomly generated example test data to help you author submission code. When your submitted notebook is scored, this example data will be replaced by the actual test data (including the sample submission).
> 
> - test/ - Folder containing randomly generated images in a format similar to the training set photos. The actual test data comprises about 6800 pet photos similar to the training set photos.
> - test.csv - Randomly generated metadata similar to the training set metadata.
> - sample_submission.csv - A sample submission file in the correct format.
>
>---

> ### **<span style="color:orange;">Photo Metadata</span>**
> The train.csv and test.csv files contain metadata for photos in the training set and test set, respectively. Each pet photo is labeled with the value of 1 (Yes) or 0 (No) for each of the following features:
> 
> - Focus - Pet stands out against uncluttered background, not too close / far.
> - Eyes - Both eyes are facing front or near-front, with at least 1 eye / pupil decently clear.
> - Face - Decently clear face, facing front or near-front.
> Near - Single pet taking up significant portion of photo (roughly over 50% of photo width or height).
> - Action - Pet in the middle of an action (e.g., jumping).
> - Accessory - Accompanying physical or digital accessory / prop (i.e. toy, digital sticker), excluding collar and leash.
> - Group - More than 1 pet in the photo.
> - Collage - Digitally-retouched photo (i.e. with digital photo frame, combination of multiple photos).
> - Human - Human in the photo.
> - Occlusion - Specific undesirable objects blocking part of the pet (i.e. human, cage or fence). Note that not all blocking objects are considered occlusion.
> - Info - Custom-added text or labels (i.e. pet name, description).
> - Blur - Noticeably out of focus or noisy, especially for the pet’s eyes and face. For Blur entries, “Eyes” column is always set to 0.
>
>---

We define a function `return_filepath` which returns the file_path of an image which we require.

In [ ]:
def return_filpath(name, folder=config.TRAIN_FOLDER_PATH):
    path = os.path.join(folder, f'{name}.jpg')
    return path

In [ ]:
train = pd.read_csv(config.TRAIN_CSV_PATH)
train['image_path'] = train['Id'].apply(lambda x: return_filpath(x))

Now we have the dataframe ready and we can move to the augmentations part.

In [ ]:
train.head()

<a id="what-is-data-augmentation"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>What is Data Augmentation?</center></h2>

## **<span style="color:orange;">Definition</span>**

Data Augmentation is a technique used to increase the amount of data by adding slightly modified copies of already existing data or newly created synthetic data from existing data.

## **<span style="color:orange;">Why is Data Augmentation Important?</span>**
Data augmentation is useful to improve performance and outcomes of machine learning models by forming new and different examples to train datasets. If dataset in a machine learning model is rich and sufficient, the model performs better and more accurate.
  
For machine learning models, collecting and labeling of data can be exhausting and costly processes. Transformations in datasets by using data augmentation techniques allow companies to reduce these operational costs.

---

## **<span style="color:orange;">How does Data Augmentation Work?</span>**

Computer vision applications use common data augmentation methods for training data. There are classic and advanced techniques in data augmentation for image recognition and natural language processing.  
  
![](https://research.aimultiple.com/wp-content/uploads/2021/04/dataaugmentaion_simpleimage_stanford-800x289.png)
  
For data augmentation, making simple alterations on visual data is popular. In addition, generative adversarial networks (GANs) are used to create new synthetic data. 
  
### **<span style="color:orange;">1. Classical Methods</span>**
**Classic image processing activities for data augmentation are:**

- Padding
- Random rotating
- Re-scaling,
- Vertical and horizontal flipping
- Translation ( image is moved along X, Y direction)
- Cropping
- Zooming
- Darkening & brightening/color modification
- Grayscaling
- Changing contrast
- Adding noise
- Random erasing

  
    
![](https://research.aimultiple.com/wp-content/uploads/2021/04/dataaugmention_image_alletranitons.png)
  
### **<span style="color:orange;">2. Advanced Methods</span>**
**Advanced models for data augmentation are:**
  
- Adversarial training/Adversarial machine learning:   
It generates adversarial examples which disrupt a machine learning model and injects them into dataset to train.
- [Generative adversarial networks (GANs)](https://research.aimultiple.com/synthetic-data/):   
GAN algorithms can learn patterns from input datasets and automatically create new examples which resemble into training data.
- [Neural style transfer](https://www.tensorflow.org/tutorials/generative/style_transfer):   Neural style transfer models can blend content image and style image and separate style from content.
- [Reinforcement learning](https://research.aimultiple.com/learning-algorithms/):    
Reinforcement learning models train software agents to reach attain their goals and make decisions in a virtual environment.
  
Popular open source python packages for data augmentation in computer vision are Keras ImageDataGenerator, Skimage and OpenCV.

---

<a id="basic-image-augmentations"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Basic Image Augmentations</center></h2>

In this section we will see what are the different **color spaces** in which we can transform the images. Based on your target you can select suitable color spaces and train model on that dataset.
  
**I will demonstrate 9 different variations of color spaces namely:**
1. [Black and White](#1.1)   
2. [Ben Graham: Greyscale + Gaussian Blur](#1.2)
3. [Hue, Saturation, Brightness](#1.3) 
4. [LUV Color Space](#1.4) 
5. [Alpha Channel](#1.5)
6. [XYZ Color Space](#1.6)
7. [Luma Chroma](#1.7)
8. [CIE Lab](#1.8)
9. [YUV Color Space](#1.9)

We shall define a simple function which will plot the augmented images for us. 

In [ ]:
def plot_augments(title, color_space):
    """
    Function which plots a 2*6 plot of images of specified color space.
    
    params: title(str)  - Title of the Plot
            color_space - color_space to which we convert the images
    """
    
    # Define subplots
    fig, axes = plt.subplots(nrows=2, ncols=6, figsize=(16,6))
    plt.suptitle(title, fontsize = 16)
    
    # Loop through images
    for i in range(0, 2*6):
        image = cv2.imread(train['image_path'][i])

        # The function converts an input image from one color space to another.
        image = cv2.cvtColor(image, color_space)
        image = cv2.resize(image, (200,200))

        x = i // 6
        y = i % 6
        
        axes[x, y].imshow(image, cmap=plt.cm.bone) 
        axes[x, y].axis('off')

## **<span style="color:orange;">Original Images</span>**
First let us plot the original images so that the augmentations become more clear to us. After this one by one we will implement different augmentations.

In [ ]:
plot_augments("Original", cv2.COLOR_BGR2RGB)

---

<a id="1.1"></a>
## **<span style="color:orange;">1. Black and White</span>**

In [ ]:
plot_augments("B&W", cv2.COLOR_RGB2GRAY)

---

<a id="1.2"></a>
## **<span style="color:orange;">2. Ben Graham: Greyscale + Gaussian Blur</span>**

As in any other signals, images also can contain different types of noise, especially because of the source (camera sensor). Image Smoothing techniques help in reducing the noise.  
  
**Gaussian filters** have the properties of having no overshoot to a step function input while minimizing the rise and fall time. In terms of image processing, any sharp edges in images are smoothed while minimizing too much blurring.
  
> OpenCV provides `cv2.gaussianblur()` function to apply Gaussian Smoothing on the input source image

### **<span style="color:orange;">#1 Without Gaussian Blur</span>**

In [ ]:
plot_augments("Without Gaussian Blur", cv2.COLOR_RGB2HSV)

### **<span style="color:orange;">#2 With Gaussian Blur</span>**

In [ ]:
plot_augments("With Gaussian Blur", cv2.COLOR_RGB2HSV)

---

<a id="1.3"></a>
## **<span style="color:orange;">3. Hue, Saturation, Brightness </span>**

- **Hue:**   
Hue, in the context of color and graphics, refers to the attribute of a visible light due to which it is differentiated from or similar to the primary colors: red, green and blue. The term is also used to refer to colors that have no added tint or shade.
  
- **Saturation:**  
Color saturation refers to the intensity of color in an image. As the saturation increases, the colors appear to be more pure. As the saturation decreases, the colors appear to be more washed-out or pale.
  
- **Brightness:**  
Brightness is the relative lightness or darkness of a particular color, from black (no brightness) to white (full brightness). Brightness is also called Lightness in some contexts
  
>`cv2.COLOR_RGB2HLS` converts `RGB/BGR` to `HLS` (hue lightness saturation) with `H` range `0..180` if `8 bit` image

In [ ]:
plot_augments("Hue, Saturation, Brightness", cv2.COLOR_RGB2HLS)

---

<a id="1.4"></a>
## **<span style="color:orange;">4. LUV Color Space</span>**

LUV decouple the "color" (chromaticity, the UV part) and "lightness" (luminance, the L part) of color. 

>Hue, Saturation, Brightness 
`cv2.COLOR_RGB2LUV` is used to convert RGB/BGR to CIE Luv.

In [ ]:
plot_augments("LUV Color Space", cv2.COLOR_RGB2LUV)

---

<a id="1.5"></a>
## **<span style="color:orange;">5. Alpha Channel</span>**
RGBA color values are an extension of RGB color values with an alpha channel - which specifies the opacity for a color.
  
An RGBA color value is specified with: `rgba(red, green, blue, alpha)`. The `alpha` parameter is a number between `0.0` (fully transparent) and `1.0` (fully opaque).
  
>`cv2.COLOR_RGB2RGBA` adds alpha channel to RGB or BGR image

In [ ]:
plot_augments("Aplha Channel", cv2.COLOR_RGB2RGBA)

---

<a id="1.6"></a>
## **<span style="color:orange;">6. XYZ Color Space</span>**

The CIE XYZ color space encompasses all color sensations that are visible to a person with average eyesight. That is why CIE XYZ (Tristimulus values) is a device-invariant representation of color.[5] It serves as a standard reference against which many other color spaces are defined.
  
>`cv2.COLOR_RGB2XYZ` convert RGB/BGR to CIE XYZ

In [ ]:
plot_augments("CIE XYZ", cv2.COLOR_RGB2XYZ)

---

<a id="1.7"></a>
## **<span style="color:orange;">7. Luma Chroma</span>**
**Chroma** is the color and the saturation of that color. **Luma** is the brightness of the pixel.
  
>`cv2.COLOR_RGB2YCrCb` convert RGB/BGR to luma-chroma (aka YCC)

In [ ]:
plot_augments("Luma Chroma", cv2.COLOR_RGB2YCrCb)

---

<a id="1.8"></a>
## **<span style="color:orange;">8. CIE Lab</span>**
The CIELAB color space also referred to as `L*a*b*`. 
  
It expresses color as three values: 
- `L*` for perceptual lightness, and 
- `a*` and `b*` for the four unique colors of human vision: red, green, blue, and yellow. 
  
CIELAB was intended as a perceptually uniform space, where a given numerical change corresponds to similar perceived change in color. While the LAB space is not truly perceptually uniform, it nevertheless is useful in industry for detecting small differences in color.
  
>`cv2.COLOR_RGB2Lab` convert RGB/BGR to CIE Lab

In [ ]:
plot_augments("CIE Lab", cv2.COLOR_RGB2Lab)

---

<a id="1.9"></a>
## **<span style="color:orange;">9. YUV Color Space</span>**

YUV is a color encoding system typically used as part of a color image pipeline. It encodes a color image or video taking human perception into account, allowing reduced bandwidth for chrominance components, thereby typically enabling transmission errors or compression artifacts to be more efficiently masked by the human perception than using a "direct" RGB-representation. 
  
>

In [ ]:
plot_augments("YUV Color Space", cv2.COLOR_RGB2YUV)

---

<a id="intermediate-image-augmentations"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Intermediate Image Augmentations</center></h2>

## **<span style="color:orange;">Torchvision Transforms</span>**

Transforms are common image transformations. They can be chained together using [`Compose`](https://pytorch.org/vision/stable/transforms.html#torchvision.transforms.Compose). 
  
Most transform classes have a function equivalent: [functional transforms](https://pytorch.org/vision/stable/transforms.html#functional-transforms) give fine-grained control over the transformations. This is useful if you have to build a more complex transformation pipeline (e.g. in the case of segmentation tasks).
  
Most transformations accept both [PIL](https://pillow.readthedocs.io/) images and tensor images, although some transformations are [PIL-only](https://pytorch.org/vision/stable/transforms.html#transforms-pil-only) and some are [tensor-only](https://pytorch.org/vision/stable/transforms.html#transforms-tensor-only). The [Conversion Transforms](https://pytorch.org/vision/stable/transforms.html#conversion-transforms) may be used to convert to and from PIL images.
  
The transformations that accept tensor images also accept batches of tensor images. A Tensor Image is a tensor with `(C, H, W)` shape, where `C` is a number of channels, `H` and `W` are image height and width. A batch of Tensor Images is a tensor of `(B, C, H, W)` shape, where B is a number of images in the batch.
  
The expected range of the values of a tensor image is implicitly defined by the tensor dtype. Tensor images with a float dtype are expected to have values in `[0, 1)`. Tensor images with an integer dtype are expected to have values in `[0, MAX_DTYPE]` where `MAX_DTYPE` is the largest value that can be represented in that dtype.

Randomized transformations will apply the same transformation to all the images of a given batch, but they will produce different transformations across calls. For reproducible transformations across calls, you may use functional transforms.

---

**In this section I will demonstrate 15 Torchvision Transforms with demos namely:**
1. [Center Crop](#2.1)
2. [Random Crop](#2.2)
3. [Random Resized Crop](#2.3)
4. [Color Jitter](#2.4)
5. [Pad](#2.5)
6. [Random Affine](#2.6)
7. [Random Horizontal Flip](#2.7)
8. [Random Vertical Flip](#2.8)
9. [Random Perspective](#2.9)
10. [Random Rotation](#2.10)
11. [Random Invert](#2.11)
12. [Random Posterize](#2.12)
13. [Random Solarize](#2.13)
14. [Random Autocontrast](#2.14)
15. [Random Equalize](#2.15)

In [ ]:
# Select a small sample of the image paths
image_list = train.sample(12)['image_path']
image_list = image_list.reset_index()['image_path']

# Show the sample
plt.figure(figsize=(16,6))
plt.suptitle("Original View", fontsize = 16)
    
for k, path in enumerate(image_list):
    image = mpimg.imread(path)
        
    plt.subplot(2, 6, k+1)
    plt.imshow(image)
    plt.axis('off') 

## **<span style="color:orange;">PyTorch Dataset Class</span>**

In [ ]:
class PetDataset(Dataset):
    def __init__(self, image_list, transforms = None):
        self.image_list = image_list
        self.transforms = transforms
        
    def __len__(self):
        return len(self.image_list)
    
    def __getitem__(self, i):
        image = plt.imread(self.image_list[i])
        image = Image.fromarray(image).convert('RGB')
        image = np.asarray(image).astype(np.uint8)
        
        if self.transforms is not None:
            image = self.transforms(image)
            
        return torch.tensor(image, dtype = torch.float)

In [ ]:
# Function to display applied transformations

def show_transform(image, title):
    """
    Function to Plot the Transformed Images
    """
    plt.figure(figsize = (16,6))
    plt.suptitle(title, fontsize = 16)
    
    #Unnormalize
    image = image / 2 + 0.5
    npimg = image.numpy()
    npimg = np.clip(npimg, 0., 1.)
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()   

In [ ]:
def apply_transform(transform_layer, title):
    """
    Function to apply Torchvision Transforms to set of Images
    
    params: transform_layer - The transform to be applied
            title (str)     - Title of the Plot
    """
    transform = transforms.Compose(
        [
            transforms.ToPILImage(),
            transforms.Resize((300 , 300)),
            transform_layer,
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])

    # Dataset
    train_dataset = PetDataset(image_list = image_list, transforms = transform)

    # DataLoader
    train_dataloader = DataLoader(dataset = train_dataset, batch_size = 12, shuffle = True)

    # Select Data
    images = next(iter(train_dataloader))

    # Show Images
    show_transform(torchvision.utils.make_grid(images, nrow = 6), title = title)

<a id="2.1"></a>
## **<span style="color:orange;">1. Center Crop</span>**
`transforms.CenterCrop` crops the given image at the center. 
  
- If the image is torch Tensor, it is expected to have `[…, H, W]` shape, where `…` means an arbitrary number of leading dimensions. 
- If image size is smaller than output size along any edge, image is padded with 0 and then center cropped.

In [ ]:
apply_transform(transforms.CenterCrop((100, 100)), "Center Crop")

---

<a id="2.2"></a>
## **<span style="color:orange;">2. Random Crop</span>**
`transforms.RandomCrop` Crop the given image at a random location. 
  
- If the image is torch Tensor, it is expected to have […, H, W] shape, where … means an arbitrary number of leading dimensions, but if non-constant padding is used, the input is expected to have at most 2 leading dimensions

In [ ]:
apply_transform(transforms.RandomCrop((100, 100)), "Random Crop")

<a id="2.3"></a>
## **<span style="color:orange;">3. Random Resized Crop</span>**
`transforms.RandomResizedCrop` Crops a random portion of image and resize it to a given size.
  
- If the image is torch Tensor, it is expected to have […, H, W] shape, where … means an arbitrary number of leading dimensions
  
A crop of the original image is made: the crop has a random area `(H * W)` and a random aspect ratio. This crop is finally resized to the given size. This is popularly used to train the **Inception networks**.

In [ ]:
apply_transform(transforms.RandomResizedCrop(size = 60), "Random Resized Crop")

---

<a id="2.4"></a>
## **<span style="color:orange;">4. Color Jitter</span>**
`transforms.ColorJitter()` randomly change the brightness, contrast, saturation and hue of an image. 
  
- If the image is torch Tensor, it is expected to have `[…, 1 or 3, H, W]` shape, where `…` means an arbitrary number of leading dimensions. 
- If img is PIL Image, mode `“1”, “I”, “F”` and modes with transparency (alpha channel) are not supported.

In [ ]:
apply_transform(transforms.ColorJitter(brightness=0.7, contrast=0.7, saturation=0.7, hue=0.5), "Color Jitter")

---

<a id="2.5"></a>
## **<span style="color:orange;">5. Pad</span>**
`transforms.Pad()` pad the given image on all sides with the given “pad” value. 
  
- If the image is torch Tensor, it is expected to have `[…, H, W]` shape, where `…` means at most 2 leading dimensions for mode reflect and symmetric, at most 3 leading dimensions for mode edge, and an arbitrary number of leading dimensions for mode constant

In [ ]:
apply_transform(transforms.Pad(padding = 5), "Padding")

---

<a id="2.6"></a>
## **<span style="color:orange;">6. Random Affine</span>**
`transforms.RandomAffine()` applies Random affine transformation of the image keeping center invariant. 
  
- If the image is torch Tensor, it is expected to have `[…, H, W]` shape, where `…` means an arbitrary number of leading dimensions.

In [ ]:
apply_transform(transforms.RandomAffine(degrees=45), "Random Affine")

---

<a id="2.7"></a>
## **<span style="color:orange;">7. Random Horizontal Flip</span>**
`transforms.RandomHorizontalFlip` Horizontally flips the given image randomly with a given probability. 
  
- If the image is torch Tensor, it is expected to have `[…, H, W]` shape, where `…` means an arbitrary number of leading dimensions

In [ ]:
apply_transform(transforms.RandomHorizontalFlip(p=0.7), "Random Horizontal Flip")

---

<a id="2.8"></a>
## **<span style="color:orange;">8. Random Vertical Flip</span>**
`transforms.RandomVerticalFlip()` Vertically flip the given image randomly with a given probability. 
  
- If the image is torch Tensor, it is expected to have `[…, H, W]` shape, where `…` means an arbitrary number of leading dimensions.

In [ ]:
apply_transform(transforms.RandomVerticalFlip(p=0.7), "Random Vertical Flip")

---

<a id="2.9"></a>
## **<span style="color:orange;">9. Random Perspective</span>**
`transforms.RandomPerspective()` Performs a random perspective transformation of the given image with a given probability. 
  
- If the image is torch Tensor, it is expected to have `[…, H, W]` shape, where `…` means an arbitrary number of leading dimensions.

In [ ]:
apply_transform(transforms.RandomPerspective(p=0.7), "Random Perspective")

<a id="2.10"></a>
## **<span style="color:orange;">10. Random Rotation</span>**  
`transforms.RandomRotation()` Rotates the image by angle. 
  
- If the image is torch Tensor, it is expected to have `[…, H, W]` shape, where `…` means an arbitrary number of leading dimensions.

In [ ]:
apply_transform(transforms.RandomRotation(degrees = 180), "Random Rotation")

---

<a id="2.11"></a>
## **<span style="color:orange;">11. Random Invert</span>**  
`transforms.RandomInvert()` Inverts the colors of the given image randomly with a given probability. 
  
- If img is a Tensor, it is expected to be in `[…, 1 or 3, H, W]` format, where `…` means it can have an arbitrary number of leading dimensions. 
- If img is PIL Image, it is expected to be in mode “L” or “RGB”.

In [ ]:
apply_transform(transforms.RandomInvert(p=0.7), "Random Invert")

---

<a id="2.12"></a>
## **<span style="color:orange;">12. Random Posterize</span>**  
`transforms.RandomPosterize()` Posterize the image randomly with a given probability by reducing the number of bits for each color channel. 
  
- If the image is torch Tensor, it should be of type `torch.uint8`, and it is expected to have `[…, 1 or 3, H, W]` shape, where `…` means an arbitrary number of leading dimensions. - If img is PIL Image, it is expected to be in mode “L” or “RGB”.

In [ ]:
apply_transform(transforms.RandomPosterize(bits = 4, p=0.7), "Random Posterize")

---

<a id="2.13"></a>
## **<span style="color:orange;">13. Random Solarize</span>** 
`transforms.RandomSolarize()` Solarizes the image randomly with a given probability by inverting all pixel values above a threshold. 
  
- If img is a Tensor, it is expected to be in `[…, 1 or 3, H, W]` format, where `…` means it can have an arbitrary number of leading dimensions. 
- If img is PIL Image, it is expected to be in mode “L” or “RGB”.

In [ ]:
apply_transform(transforms.RandomSolarize(threshold = 30, p=0.7), "Random Solarize")

---

<a id="2.14"></a>
## **<span style="color:orange;">14. Random Auto Contrast</span>** 
`transforms.RandomAutocontrast()` Autocontrasts the pixels of the given image randomly with a given probability. 
  
- If the image is torch Tensor, it is expected to have `[…, 1 or 3, H, W]` shape, where `…` means an arbitrary number of leading dimensions. 
- If img is PIL Image, it is expected to be in mode “L” or “RGB”.

In [ ]:
apply_transform(transforms.RandomAutocontrast(p=0.7), "Random AutoContrast")

---

<a id="2.15"></a>
## **<span style="color:orange;">15. Random Equalize</span>** 
`transforms.RandomEqualize()` Equalizes the histogram of the given image randomly with a given probability. 
  
- If the image is torch Tensor, it is expected to have `[…, 1 or 3, H, W]` shape, where `…` means an arbitrary number of leading dimensions. 
- If img is PIL Image, it is expected to be in mode “P”, “L” or “RGB”.

In [ ]:
apply_transform(transforms.RandomEqualize(p=0.7), "Random Equalize")

---

<a id="advanced-data-augmentations"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Advanced Data Augmentations</center></h2>

The idea is to generate new and realistic fetures based on labels. GANs are excellent at generating realistic data. We can condition this generation by using [Conditional Generative Adversarial Networks](https://arxiv.org/abs/1411.1784)

In [ ]:
latent_dim = 100 # dimension of the latent space
n_samples = 1000 # size of our dataset
n_classes = 3
n_features = 2 # we use 2 features since we'd like to visualize them

We start by creating random clusters of points, n_classes, with features, n_features. We make use of make_blobs from scikit learn that generates gaussian blobs

In [ ]:
from sklearn.datasets import make_blobs

X, y = make_blobs(n_samples=n_samples, centers=n_classes, n_features=n_features, random_state=123)

print('Size of our dataset:', len(X))
print('Number of features:', X.shape[1])
print('Classes:', set(y))

Following we normalize our features to help with the learning

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(-1, 1))

scaled_X = scaler.fit_transform(X)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 4))
legend = []

for i in range(n_classes):
    plt.scatter(scaled_X[:, 0][np.where(y==i)], scaled_X[:, 1][np.where(y==i)], )
    legend.append('Class %d' % i)

ax.set_xlabel('Feature 1')
ax.set_ylabel('Feature 2')
ax.legend(legend)

In [ ]:
def build_discriminator(optimizer=Adam(0.0002, 0.5)):
    '''
    Defines and compiles discriminator model.
    This architecture has been inspired by:
    https://github.com/eriklindernoren/Keras-GAN/blob/master/cgan/cgan.py
    and adapted for this problem.
    
    Params:
        optimizer=Adam(0.0002, 0.5) - recommended values
    '''
    features = Input(shape=(n_features,))
    label = Input(shape=(1,), dtype='int32')
    
    # Using an Embedding layer is recommended by the papers
    label_embedding = Flatten()(Embedding(n_classes, n_features)(label))
    
    # We condition the discrimination of generated features 
    inputs = multiply([features, label_embedding])
    
    x = Dense(512)(inputs)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dense(512)(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dropout(0.4)(x)
    x = Dense(512)(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dropout(0.4)(x)
    
    valid = Dense(1, activation='sigmoid')(x)
    
    model = Model([features, label], valid)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    model.summary()

    return model

In [ ]:
def build_generator():
    '''
    Defines the generator model.
    This architecture has been inspired by:
    https://github.com/eriklindernoren/Keras-GAN/blob/master/cgan/cgan.py
    and adapted for this problem.
    '''
    
    noise = Input(shape=(latent_dim,))
    label = Input(shape=(1,), dtype='int32')
    
    # Using an Embedding layer is recommended by the papers
    label_embedding = Flatten()(Embedding(n_classes, latent_dim)(label))
    
    # We condition the generation of features
    inputs = multiply([noise, label_embedding])
    
    x = Dense(256)(inputs)
    x = LeakyReLU(alpha=0.2)(x)
    x = BatchNormalization(momentum=0.8)(x)
    x = Dense(512)(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = BatchNormalization(momentum=0.8)(x)
    x = Dense(1024)(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = BatchNormalization(momentum=0.8)(x)
    
    features = Dense(n_features, activation='tanh')(x)
    
    model = Model([noise, label], features)
    model.summary()

    return model

In [ ]:
def build_gan(generator, discriminator, optimizer=Adam(0.0002, 0.5)):
    '''
    Defines and compiles GAN model. It bassically chains Generator
    and Discriminator in an assembly-line sort of way where the input is
    the Generator's input. The Generator's output is the input of the Discriminator,
    which outputs the output of the whole GAN.
    
    Params:
        optimizer=Adam(0.0002, 0.5) - recommended values
    '''
    
    noise = Input(shape=(latent_dim,))
    label = Input(shape=(1,))
    
    features = generator([noise, label])
    valid = discriminator([features, label])
    
    # We freeze the discriminator's layers since we're only 
    # interested in the generator and its learning
    discriminator.trainable = False
    
    model = Model([noise, label], valid)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    model.summary()
    
    return model

In [ ]:
discriminator = build_discriminator()

In [ ]:
generator = build_generator()

In [ ]:
gan = build_gan(generator, discriminator)

In [ ]:
def get_random_batch(X, y, batch_size):
    '''
    Will return random batches of size batch_size
    
    Params:
        X: numpy array - features
        y: numpy array - classes
        batch_size: Int
    '''
    idx = np.random.randint(0, len(X))
    
    X_batch = X[idx:idx+batch_size]
    y_batch = y[idx:idx+batch_size]
    
    return X_batch, y_batch

In [ ]:
def train_gan(gan, generator, discriminator, 
              X, y, 
              n_epochs=100, batch_size=32, 
              hist_every=10, log_every=1):
    '''
    Trains discriminator and generator (last one through the GAN) 
    separately in batches of size batch_size. The training goes as follow:
        1. Discriminator is trained with real features from our training data
        2. Discriminator is trained with fake features generated by the Generator
        3. GAN is trained, which will only change the Generator's weights.
        
    Params:
        gan: GAN model
        generator: Generator model
        discriminator: Discriminator model
        X: numpy array - features
        y: numpy array - classes
        n_epochs: Int
        batch_size: Int
        hist_every: Int - will save the training loss and accuracy every hist_every epochs
        log_every: Int - will output the loss and accuracy every log_every epochs
    
    Returns:
        loss_real_hist: List of Floats
        acc_real_hist: List of Floats
        loss_fake_hist: List of Floats
        acc_fake_hist: List of Floats
        loss_gan_hist: List of Floats
        acc_gan_hist: List of Floats
    '''
    
    half_batch = int(batch_size / 2)
    
    acc_real_hist = []
    acc_fake_hist = []
    acc_gan_hist = []
    loss_real_hist = []
    loss_fake_hist = []
    loss_gan_hist = []
    
    # Initialize W&B
    run = wandb.init(project='PetFinder-GANs', 
                 config= WANDB_CONFIG)
    
    for epoch in range(n_epochs):
        
        X_batch, labels = get_random_batch(X, y, batch_size)
        
        # train with real values
        y_real = np.ones((X_batch.shape[0], 1))
        loss_real, acc_real = discriminator.train_on_batch([X_batch, labels], y_real)
        
        # train with fake values
        noise = np.random.uniform(0, 1, (labels.shape[0], latent_dim))
        X_fake = generator.predict([noise, labels])
        y_fake = np.zeros((X_fake.shape[0], 1))
        loss_fake, acc_fake = discriminator.train_on_batch([X_fake, labels], y_fake)
        
        y_gan = np.ones((labels.shape[0], 1))
        loss_gan, acc_gan = gan.train_on_batch([noise, labels], y_gan)
        
        if (epoch+1) % hist_every == 0:
            acc_real_hist.append(acc_real)
            acc_fake_hist.append(acc_fake)
            acc_gan_hist.append(acc_gan)
            loss_real_hist.append(loss_real)
            loss_fake_hist.append(loss_fake)
            loss_gan_hist.append(loss_gan)
            
            wandb.log({'acc_real': acc_real,
                      'acc_fake': acc_fake,
                      'acc_gan': acc_gan,
                      'loss_real': loss_real,
                      'loss_fake': loss_fake,
                      'loss_gan': loss_gan,
                     })

        if (epoch+1) % log_every == 0:
            lr = 'loss real: {:.3f}'.format(loss_real)
            ar = 'acc real: {:.3f}'.format(acc_real)
            lf = 'loss fake: {:.3f}'.format(loss_fake)
            af = 'acc fake: {:.3f}'.format(acc_fake)
            lg = 'loss gan: {:.3f}'.format(loss_gan)
            ag = 'acc gan: {:.3f}'.format(acc_gan)

            print('{}, {} | {}, {} | {}, {}'.format(lr, ar, lf, af, lg, ag))
    
    # Close W&B run
    wandb.finish()
        
    return loss_real_hist, acc_real_hist, loss_fake_hist, acc_fake_hist, loss_gan_hist, acc_gan_hist

In [ ]:
loss_real_hist, acc_real_hist, loss_fake_hist, acc_fake_hist, loss_gan_hist, acc_gan_hist = train_gan(gan, generator, discriminator, scaled_X, y)

## [Check out the run page here $\rightarrow$](https://wandb.ai/ishandutta/PetFinder-GANs?workspace=user-ishandutta)

In [ ]:
def generate_samples(class_for, n_samples=20):
    '''
    Generates new random but very realistic features using
    a trained generator model
    
    Params:
        class_for: Int - features for this class
        n_samples: Int - how many samples to generate
    '''
    
    noise = np.random.uniform(0, 1, (n_samples, latent_dim))
    label = np.full((n_samples,), fill_value=class_for)
    return generator.predict([noise, label])

In [ ]:
features_class_0 = generate_samples(0)

In [ ]:
def visualize_fake_features(fake_features, figsize=(15, 6), color='r'):
    ax, fig = plt.subplots(figsize=figsize)
    
    # Let's plot our dataset to compare
    for i in range(n_classes):
        plt.scatter(scaled_X[:, 0][np.where(y==i)], scaled_X[:, 1][np.where(y==i)])

    plt.scatter(fake_features[:, 0], fake_features[:, 1], c=color)
    plt.title('Real and fake features')
    plt.legend(['Class 0', 'Class 1', 'Class 2', 'Fake'])

In [ ]:
visualize_fake_features(features_class_0)

In [ ]:
features_class_1 = generate_samples(1)
visualize_fake_features(features_class_1)

<a id="references"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>References</center></h2>

>- [What is Data Augmentation? Techniques, Benefit and Examples](https://research.aimultiple.com/data-augmentation/)
>- [🧬SIIM Melanoma Competition: EDA + Augmentations](https://www.kaggle.com/andradaolteanu/siim-melanoma-competition-eda-augmentations)
>- [Data Augmentation using Conditional GAN (cGAN)](https://medium.com/@jscriptcoder/data-augmentation-using-conditional-gan-cgan-d5e8d33ad032)
>
>---

<h1><center>More Plots coming soon!</center></h1>
                                                      
<center><img src = "https://static.wixstatic.com/media/5f8fae_7581e21a24a1483085024f88b0949a9d~mv2.jpg/v1/fill/w_934,h_379,al_c,q_90/5f8fae_7581e21a24a1483085024f88b0949a9d~mv2.jpg" width = "750" height = "500"/></center> 

<div class="list-group" id="list-tab" role="tablist">
<h3 class="list-group-item list-group-item-action active" data-toggle="list" style='background:maroon; border:0; color:white' role="tab" aria-controls="home"><center>If you find this notebook useful, do give me an upvote, it helps to keep up my motivation. This notebook will be updated frequently so keep checking for furthur developments.</center></h3>

--- 

## **<span style="color:orange;">Let's have a Talk!</span>**
> ### Reach out to me on [LinkedIn](https://www.linkedin.com/in/ishandutta0098)

---